In [1]:
import pandas as pd

import aidd.sys.config as cfg
from aidd.utils.data_io import read_data, save_data
from aidd.utils.data_io import read_pickle, save_pickle

In [2]:
def service(row_df=None, scaler=None):
    df = row_df
    pc = df.loc[0, 'POLE_CNT']
    if pc == 1.0:
        processing_scaler = scaler['1']
    else:
        processing_scaler = scaler['N1']
    all_scaler = scaler['ALL']
    scaled_data = processing_scaler.transform(df)
    scaled_all = all_scaler.transform(df)
    
    # print(scaled_data[0], scaled_all[0])
    
    tcc = 0
    return tcc

In [3]:
def service_main(pdf=None):
    df = read_data('ONLINE') if pdf is None else pdf
    # 추후에 메모리에 상주한 클레스의 인스턴스에 로드할 것
    scaler = {pc_mode: read_pickle(f'DUMP,SCALER,{pc_mode}') for pc_mode in cfg.DATA_PC_TYPE}
    
    ### 임시 체크 코드
    pb_df = read_data('PP,SL', dtype={'CONS_ID': str})
    bcols = pb_df.columns.tolist()
    ocols = df.columns.tolist()
    for i in range(len(bcols)):
        if bcols[i] != ocols[i]:
            print('False')
    ### 임시 체크 코드
    
    # 리턴할 데이터프레임
    return_df = df[['CONS_ID']].copy()
    return_tcc = []
    
    training_cols = df.columns[5:].tolist()
    X = df[training_cols].copy()
    for _, row in X.iterrows():
        row_df = pd.DataFrame(row).transpose()
        # 아래 코드를 사용해야 service 함수에서 0번 인덱스를 사용할 수 없음
        row_df.reset_index(drop=True, inplace=True)
        return_tcc.append(service(row_df, scaler))
        
    return_df.loc[:, 'TOTAL_CONS_COST'] = return_tcc
    return return_df

In [4]:
rdf = service_main()
rdf

,CONS_ID,TOTAL_CONS_COST
0,477420204194,0
1,475920223725,0
2,474620226651,0


#### 여기서부터는 단순 테스트

In [5]:
pb_df = read_data('PP,SL')
po_df = read_data('ONLINE')

/home/freeman/projects/aidd/utils/data_io.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, **kwargs)


In [6]:
bcols = pb_df.columns.tolist()
ocols = po_df.columns.tolist()

In [7]:
for i in range(len(bcols)):
    if bcols[i] != ocols[i]:
        print('False')